In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
driver_options = Options()
driver_options.add_argument("accept-language=en-US")

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s,options=driver_options)
driver.maximize_window()

actions = ActionChains(driver)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\emman\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [3]:
INITIAL_WEBSITE = "https://www.jumia.sn/"
data = {}
# go to the root page and get header/menu with its sub-items

In [4]:
def process_article(article):
    """retrive needed information from an article"""
    try:
        name = article.find_element(by=By.CSS_SELECTOR, value='h3.name').text
        price = article.find_element(by=By.CSS_SELECTOR, value='div.prc').text
        img_link = article.find_element(by=By.CSS_SELECTOR, value='img.img').get_attribute('data-src')
        descrption_link = article.find_element(by=By.CSS_SELECTOR, value='a.core').get_attribute('href')
        try:
            old_price, decrease_percentage = article.find_element(by=By.CSS_SELECTOR, value='div.s-prc-w').text.split("\n")
        except:
            old_price, decrease_percentage = '', ''
        try:
            n_stars, n_votes = article.find_element(by=By.CSS_SELECTOR, value='div.rev').text.split("\n")
            # n_votes = int(n_votes.replace("(", "").replace(")", "")) # clean value and convert to int 
        except:
            n_stars, n_votes = '', ''
        try:
            article.find_element(by=By.CSS_SELECTOR, value='svg.ic.xprss')
            jumia_express = True
        except:
            jumia_express = False

        article_data = dict(name=name , price=price , descrption_link=descrption_link , img_link=img_link , jumia_express=jumia_express ,
                       old_price=old_price , decrease_percentage=decrease_percentage ,n_stars=n_stars, n_votes=n_votes  )
    except:
        article_data = None
    return article_data

In [5]:
def close_popup(driver):
    """close pop up frames"""
    try:
        [btn.click() for btn in driver.find_elements(by=By.CSS_SELECTOR, value='button.cls')]
    except:
        pass

In [ ]:
driver.get(INITIAL_WEBSITE)
close_popup(driver)

pages = []
categories = driver.find_elements(by=By.CSS_SELECTOR, value='div.flyout-w > div > a.itm')
for (i,category) in enumerate(categories):
    actions.move_to_element(category).perform()
    sub_menu = driver.find_elements(by=By.CSS_SELECTOR, value='div.flyout-w > div > div.sub')[i]
    sub_menu_groups = sub_menu.find_elements(by=By.CSS_SELECTOR, value='div > div > div.cat')
    # print(f"category : {category.text} \n")
    for sub_menu_group in sub_menu_groups:
        sub_menu_group_name = sub_menu_group.find_element(by=By.CSS_SELECTOR, value='a.tit').text
        # print(f"sub-category: {sub_menu_group_name} ")
        sub_menu_group_items = sub_menu_group.find_elements(by=By.CSS_SELECTOR, value='a.s-itm')
        for item in sub_menu_group_items:
            link = item.get_attribute('href')
            if link and str(link).strip() != '':
                if str(item.text).strip() != '':
                    pages.append( { 'link': link, 'category':category.text, 'sub_category': sub_menu_group_name,
                                   'section': item.text.lower(),} )
                else:
                    section = link[:-1].split('/')[-1].replace("-", " ").lower()
                    pages.append( { 'link': link, 'category':category.text, 'sub_category': sub_menu_group_name,
                                   'section': section,} )
        #         print(f"section : {item.text} , {link}   ") 
        # print('')
    
print(f"Available sections : {len(pages)} \n" ) 

for page in pages:
    category_name, category_link = page['section'], page['link']
    print(f"{category_name} -- {category_link} " )
    if category_link is not None :
        last_page_scraped = False
        next_page = category_link

        while not last_page_scraped:
            # go to a page to scrape articles
            driver.get(next_page)
            close_popup(driver)
            articles_ = driver.find_elements(by=By.CSS_SELECTOR, value='article.prd._fb.col.c-prd') # get article boxes
            articles = [ ]
            for article in articles_:
                article_data = process_article(article)
                if article_data:
                    articles.append(article_data)
            data[category_name] = articles
            try:
                next_page_btn = driver.find_elements(by=By.CSS_SELECTOR, value='div.pg-w.-ptm.-pbxl a')[-2]
                if next_page_btn.accessible_name not in ['Page suivante']:
                    last_page_scraped = True
                else:
                    next_page = next_page_btn.get_attribute("href")
            except:
                last_page_scraped = True

            break # uncomment if you just want to get first page articles
        try:
            n_results_str = driver.find_element(by=By.CSS_SELECTOR, value='p.-gy5.-phs').text
        except:
            n_results_str = '0 results'
        print(f"Available articles in this category : {n_results_str.split()[0]} \n")
    
    # break # uncomment if you just want to get first category articles

Available sections : 252 

smartphones -- https://www.jumia.sn/smartphones/ 
Available articles in this category : 774 

téléphones basics -- https://www.jumia.sn/telephones-portables/ 
Available articles in this category : 169 

accessoires bluetooth -- https://www.jumia.sn/bluetooth/ 
Available articles in this category : 103 

batteries téléphones -- https://www.jumia.sn/telephone-batteries/ 
Available articles in this category : 51 

cartes microsd -- https://www.jumia.sn/telephones-tablettes-cartes-microsd/ 
Available articles in this category : 18 

cartes mémoire -- https://www.jumia.sn/telephones-cartes-memoires/ 
Available articles in this category : 14 

chargeurs & adaptateurs -- https://www.jumia.sn/telephonie-chargeurs-adaptateurs/ 
Available articles in this category : 15 

chargeurs téléphones -- https://www.jumia.sn/telephone-chargeur/ 
Available articles in this category : 71 

coques -- https://www.jumia.sn/telephones-protection/ 
Available articles in this category :

In [ ]:
list_of_dfs = []
for section, articles in data.items():
    df = pd.DataFrame(articles)
    df.to_csv( os.path.join( "data", f"section {category}.csv"), index=False  )
    
    df['section'] = section
    list_of_dfs.append(df)
    

global_df = pd.concat( list_of_dfs, axis=0)
global_df.to_csv( os.path.join( "data", f"GLOBAL_SECTION.csv"), index=False  )